In [19]:
import pysurvival
from pysurvival.models.simulations import SimulationModel
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 
import torch # For building the networks 
import torchtuples as tt # Some useful functions
from torch import nn

from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

from torch.utils.data import DataLoader

from stg import STG,meter
import stg.utils as stg_utils
from stg.utils import SimpleDataset,FastTensorDataLoader,as_numpy,as_tensor

import numpy as np
import torch
import time
from DeepSurv.deepsurv import deep_surv,utils,viz
from DeepSurv.deepsurv.deepsurv_logger import DeepSurvLogger, TensorboardLogger

import numpy as np
import pandas as pd

import lasagne
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
)

from sksurv.linear_model import CoxPHSurvivalAnalysis
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../auton-survival/')

from auton_survival import DeepCoxPH
from tqdm import tqdm

In [2]:
#datasets = utils.load_cox_gaussian_data()

def minmax_scale(df):
    scaler = MinMaxScaler()
    scaler.fit(df)
    return scaler.transform(df)


def minmax_stand_dfs(train_data,test_data,val_data):
    
    features = list(train_data.columns)
    features.remove("time")
    features.remove("event")
    
    train_data[features] = minmax_scale(train_data[features])
    test_data[features] = minmax_scale(test_data[features])
    val_data[features] = minmax_scale(val_data[features])
    
    return train_data,test_data,val_data
    
    

def dataframe_to_datadicts(df, event_col = 'event', time_col = 'time'):
    # Extract the event and time columns as numpy arrays
    e = df[event_col].values.astype(np.int32)
    t = df[time_col].values.astype(np.float32)

    # Extract the patient's covariates as a numpy array
    x_df = df.drop([event_col, time_col], axis = 1)
    x = x_df.values.astype(np.float32)
    
    # Return the deep surv dataframe
    return {
        'x' : x,
        'e' : e,
        't' : t
    }

def prepare_stg_dataset(train_dict,valid_dict,test_dict):
    
    train_X = train_dict['x']
    train_y = {'e': train_dict['e'], 't': train_dict['t']}
    valid_X = valid_dict['x']
    valid_y = {'e': valid_dict['e'], 't': valid_dict['t']}
    test_X = test_dict['x']
    test_y = {'e': test_dict['e'], 't': test_dict['t']}

    train_dict={}
    train_dict['X'], train_dict['E'], train_dict['T'] = stg_utils.prepare_data(train_X, train_y)
    train_dict['ties'] = 'noties'

    valid_dict={}
    valid_dict['X'], valid_dict['E'], valid_dict['T'] = stg_utils.prepare_data(valid_X, valid_y)
    valid_dict['ties'] = 'noties'

    test_dict = {}
    test_dict['X'], test_dict['E'], test_dict['T'] = stg_utils.prepare_data(test_X, test_y)
    test_dict['ties'] = 'noties'
    
    return train_dict,test_dict,valid_dict


def make_prediction_stg(stg_dict, stg_model):

    data_loader = FastTensorDataLoader(torch.from_numpy(stg_dict["X"]).float().to(stg_model.device), 
                        tensor_names=('X'), shuffle=False)
    res = []
    stg_model._model.eval()
    for feed_dict in data_loader:
        feed_dict_np = as_numpy(feed_dict)
        feed_dict = as_tensor(feed_dict)
        with torch.no_grad():
            output_dict = stg_model._model(feed_dict)
            output_dict_np = as_numpy(output_dict)
            res.append(output_dict_np['logits'])
    return np.concatenate(res, axis=0)

def evaluate_sklearn_methods(train_data,test_data,sklearn_estimator,data_model_name):
    X_train = train_data.drop(["time","event"],axis=1).values
    X_test = test_data.drop(["time","event"],axis=1).values
    
    y_train_pred = sklearn_estimator.predict(X_train)
    y_test_pred = sklearn_estimator.predict(X_test)
    
    conc_train = concordance_index_censored(train_data["event"].astype(bool).values,
                                           train_data["time"].values,y_train_pred)[0]
    conc_test = concordance_index_censored(test_data["event"].astype(bool).values,
                                           test_data["time"].values,y_test_pred)[0]
    
    sklearn_metrics = pd.DataFrame({
                                (conc_train,
                                 conc_test
                                 )}
                                 ,columns=["train_conc","test_conc"],index=[data_model_name])
    return sklearn_metrics
    

def evaluate_stg_model(stg_train_data, stg_val_data,stg_test_data,stg_model,data_set_name):
    
    train_pred_stg = make_prediction_stg(stg_train_data,stg_model)
    val_pred_stg = make_prediction_stg(stg_val_data,stg_model)
    test_pred_stg = make_prediction_stg(stg_test_data,stg_model)

    con_ind_train = concordance_index_censored(stg_train_data["E"].astype(bool),stg_train_data["T"].astype(bool),
                                           train_pred_stg.ravel())[0]
    con_ind_test = concordance_index_censored(stg_test_data["E"].astype(bool),stg_test_data["T"].astype(bool),
                                           test_pred_stg.ravel())[0]
    con_ind_val = concordance_index_censored(stg_val_data["E"].astype(bool),stg_val_data["T"].astype(bool),
                                           val_pred_stg.ravel())[0]

    stg_metrics = pd.DataFrame({
                                (con_ind_train,
                                 con_ind_val,
                                 con_ind_test)}
                                 ,columns=["train_conc","val_con","test_conc"],index=[data_set_name])
    return stg_metrics


def evaluate_ds_model(train_dict,val_dict,test_dict,ds_model,data_set_name):
    y_pred_train_ds = ds_model.predict_risk(train_dict["x"])
    y_pred_val_ds = ds_model.predict_risk(val_dict["x"])
    y_pred_test_ds = ds_model.predict_risk(test_dict["x"])

    con_ind_train = concordance_index_censored(train_dict["e"].astype(bool),train_dict["t"].astype(bool),
                                           y_pred_train_ds.ravel())[0]
    con_ind_test = concordance_index_censored(test_dict["e"].astype(bool),test_dict["t"].astype(bool),
                                           y_pred_test_ds.ravel())[0]
    con_ind_val = concordance_index_censored(val_dict["e"].astype(bool),val_dict["t"].astype(bool),
                                           y_pred_val_ds.ravel())[0]

    ds_metrics = pd.DataFrame({
                                (con_ind_train,
                                 con_ind_val,
                                 con_ind_test)}
                                 ,columns=["train_conc","val_con","test_conc"],index=[data_set_name])
    return ds_metrics


## Data Preparation

In [3]:
%pylab inline

# Initializing the simulation model
lin_sim = SimulationModel(survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1.0,
                       alpha = 0.01,
                       beta = 5., )

gau_sim = SimulationModel(survival_distribution = 'exponential',
                       risk_type = 'gaussian',
                       censored_parameter = 7.0,
                       alpha = 0.01,
                       beta = 5., )

# Generating N Random samples
N = 5000
linear_data_train=lin_sim.generate_data(num_samples = N, num_features=2).astype(float32)
linear_data_test=lin_sim.generate_data(num_samples = int(0.2*N), num_features=2).astype(float32)
linear_data_val=lin_sim.generate_data(num_samples = int(0.2*N), num_features=2).astype(float32)

gaussian_data_train = gau_sim.generate_data(num_samples = N, num_features=2)
gaussian_data_test = gau_sim.generate_data(num_samples = int(0.2*N), num_features=2)
gaussian_data_val = gau_sim.generate_data(num_samples = int(0.2*N), num_features=2)

linear_data_train,linear_data_test,linear_data_val =  minmax_stand_dfs(linear_data_train,linear_data_test,linear_data_val)
gaussian_data_train,gaussian_data_test,gaussian_data_val = minmax_stand_dfs(gaussian_data_train,gaussian_data_test,gaussian_data_val)

lin_train_dict = dataframe_to_datadicts(linear_data_train)
lin_test_dict = dataframe_to_datadicts(linear_data_test)
lin_val_dict = dataframe_to_datadicts(linear_data_val)

gau_train_dict = dataframe_to_datadicts(gaussian_data_train)
gau_test_dict = dataframe_to_datadicts(gaussian_data_test)
gau_val_dict = dataframe_to_datadicts(gaussian_data_val)

Populating the interactive namespace from numpy and matplotlib
Number of data-points: 5000 - Number of events: 280.0
Number of data-points: 1000 - Number of events: 55.0
Number of data-points: 1000 - Number of events: 57.0
Number of data-points: 5000 - Number of events: 534.0
Number of data-points: 1000 - Number of events: 114.0
Number of data-points: 1000 - Number of events: 116.0


In [4]:
stg_train_lin,stg_test_lin,stg_val_lin = prepare_stg_dataset(lin_train_dict,lin_test_dict,lin_val_dict)
stg_train_gau,stg_test_gau,stg_val_gau = prepare_stg_dataset(gau_train_dict,gau_test_dict,gau_val_dict)


## Baselines With Sklearn Implementations

In [5]:
sklearn_train_lin = pd.concat([linear_data_train,linear_data_val]).copy()
sklearn_test_lin = linear_data_test.copy()

X_lin = sklearn_train_lin.drop(["time","event"],axis=1).values
y_lin = np.array(sklearn_train_lin[["event","time"]].apply(tuple,axis=1).values,
             dtype=[('Status', '?'), ('Survival_in_days', '<f8')])


In [6]:
cox_est = CoxPHSurvivalAnalysis().fit(X_lin,y_lin)
evaluate_sklearn_methods(sklearn_train_lin,sklearn_test_lin,cox_est,"cox_lin")

,train_conc,test_conc
cox_lin,0.845814,0.819414


In [7]:
rf_estimator = RandomSurvivalForest().fit(X_lin,y_lin)
evaluate_sklearn_methods(sklearn_train_lin,sklearn_test_lin,rf_estimator,"rf_lin")

,train_conc,test_conc
rf_lin,0.981426,0.793979


In [8]:
sklearn_train_gau = pd.concat([gaussian_data_train,gaussian_data_val]).copy()
sklearn_test_gau = gaussian_data_val.copy()

X_gau = sklearn_train_gau.drop(["time","event"],axis=1).values
y_gau = np.array(sklearn_train_gau[["event","time"]].apply(tuple,axis=1).values,
             dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

In [9]:
cox_est_gau = CoxPHSurvivalAnalysis().fit(X_gau,y_gau)
evaluate_sklearn_methods(sklearn_train_gau,sklearn_test_gau,cox_est_gau,"cox_gau")

,train_conc,test_conc
cox_gau,0.515443,0.502828


In [10]:
rf_estimator_gau = RandomSurvivalForest().fit(X_gau,y_gau)
evaluate_sklearn_methods(sklearn_train_gau,sklearn_test_gau,rf_estimator_gau,"rf_gau")

,train_conc,test_conc
rf_gau,0.935838,0.966817


## STG on Linear Data

In [47]:
device = "cpu" 
feature_selection = True

lin_stg_model = STG(task_type='cox',input_dim=stg_train_lin['X'].shape[1], output_dim=1, hidden_dims=[25, 25], activation='selu',
    optimizer='Adam', learning_rate=1e-03, batch_size=stg_train_lin['X'].shape[0], feature_selection=feature_selection, 
    sigma=0.5, lam=0.004, random_state=1, device=device)

now = time.time()
lin_stg_model.fit(stg_train_lin['X'], {'E': stg_train_lin['E'], 'T': stg_train_lin['T']}, nr_epochs=200, 
        valid_X=stg_val_lin['X'], valid_y={'E': stg_val_lin['E'], 'T': stg_val_lin['T']}, print_interval=100)
print("Passed time: {}".format(time.time() - now))

Epoch: 100: CI=0.832527 loss=8.246545 valid_CI=0.829125 valid_loss=5.348845
Epoch: 200: CI=0.796939 loss=8.028740 valid_CI=0.815529 valid_loss=5.098709
Passed time: 4.390125036239624


In [48]:
lin_stg_metrics = evaluate_stg_model(stg_train_lin,stg_val_lin,stg_test_lin,lin_stg_model,"stg_lin")
lin_stg_metrics

,train_conc,val_con,test_conc
stg_lin,0.839897,0.810805,0.831281


In [63]:
lin_stg_model._model

STGCoxModel(
  (mlp): Sequential(
    (0): LinearLayer(
      (0): Linear(in_features=2, out_features=25, bias=True)
      (1): SELU(inplace=True)
    )
    (1): LinearLayer(
      (0): Linear(in_features=25, out_features=25, bias=True)
      (1): SELU(inplace=True)
    )
    (2): Linear(in_features=25, out_features=1, bias=True)
  )
  (FeatureSelector): FeatureSelector()
)

## STG on Nonlinear Data

In [61]:
device = "cpu" 
feature_selection = False 

gau_stg_model = STG(task_type='cox',input_dim=stg_train_gau['X'].shape[1], output_dim=1, hidden_dims=[25,25,5], activation='ReLU',
    optimizer='Adam', learning_rate=1e-03, batch_size=stg_train_gau['X'].shape[0], feature_selection=feature_selection, 
    sigma=0.5, lam=10, random_state=1, device=device)

now = time.time()
gau_stg_model.fit(stg_train_gau['X'], {'E': stg_train_gau['E'], 'T': stg_train_gau['T']}, nr_epochs=300, 
        valid_X=stg_val_gau['X'], valid_y={'E': stg_val_gau['E'], 'T': stg_val_gau['T']}, print_interval=100)
print("Passed time: {}".format(time.time() - now))

Epoch: 100: CI=0.538677 loss=7.654104 valid_CI=0.504208 valid_loss=5.985466
Epoch: 200: CI=0.547508 loss=7.654143 valid_CI=0.546263 valid_loss=5.985450
Epoch: 300: CI=0.517855 loss=7.654148 valid_CI=0.505034 valid_loss=5.985452
Passed time: 8.290906190872192


In [62]:
evaluate_stg_model(stg_train_gau,stg_val_gau,stg_test_gau,gau_stg_model,"gau_stg")

,train_conc,val_con,test_conc
gau_stg,0.511045,0.497562,0.509096


## DeepSurv Model Linear

In [15]:
deep_surv_hyperparams_lin = {
    'L2_reg': 10.0,
    'batch_norm': True,
    'dropout': 0.1,
    'hidden_layers_sizes': [25, 25],
    'learning_rate': 1e-03,
    'lr_decay': 0.01,
    'momentum': 0.1,
    'n_in': lin_train_dict['x'].shape[1],
    'standardize': True
}

In [24]:
# Create an instance of DeepSurv using the hyperparams defined above
ds_lin = deep_surv.DeepSurv(**deep_surv_hyperparams_lin)
logger = None


# Now we train the model
update_fn=lasagne.updates.nesterov_momentum 
                                            
n_epochs = 2000

# If you have validation data, you can add it as the second parameter to the function
lin_model_params = ds_lin.train(lin_train_dict,lin_val_dict, n_epochs=n_epochs, logger=logger, update_fn=update_fn)

In [27]:
evaluate_ds_model(lin_train_dict,lin_val_dict,lin_test_dict,ds_lin,"ds_lin")

,train_conc,val_con,test_conc
ds_lin,0.819997,0.859606,0.831108


## Deep Surv Model Nonlinear

In [28]:
deep_surv_hyperparams_gau = {
    'L2_reg': 10.0,
    'batch_norm': True,
    'dropout': 0.1,
    'hidden_layers_sizes': [25, 25],
    'learning_rate': 1e-03,
    'lr_decay': 0.01,
    'momentum': 0.1,
    'n_in': lin_train_dict['x'].shape[1],
    'standardize': True
}

In [29]:
# Create an instance of DeepSurv using the hyperparams defined above
ds_gau = deep_surv.DeepSurv(**deep_surv_hyperparams_gau)
logger = None


# Now we train the model
update_fn=lasagne.updates.nesterov_momentum 
                                            
n_epochs = 2500

# If you have validation data, you can add it as the second parameter to the function
gau_model_params = ds_gau.train(gau_train_dict,gau_val_dict, n_epochs=n_epochs, logger=logger, update_fn=update_fn)

In [30]:
evaluate_ds_model(gau_train_dict,gau_val_dict,gau_test_dict,ds_gau,"gau_lin")

,train_conc,val_con,test_conc
gau_lin,0.494497,0.477374,0.465217


## Auton CoxPH

In [5]:
lin_train_dict["e"]

array([0, 1, 0, ..., 0, 0, 0], dtype=int32)

In [6]:
lincoxph = DeepCoxPH()
lincoxph.fit(lin_train_dict["x"], lin_train_dict["t"], lin_train_dict["e"],iters = 500)
risk_preds = lincoxph.torch_model[0](torch.from_numpy(lin_train_dict["x"]))
risk_preds = risk_preds.detach().numpy()
concordance_index_censored(lin_train_dict["e"].astype(bool),lin_train_dict["t"],risk_preds.ravel())

 59%|█████▉    | 296/500 [00:05<00:03, 51.65it/s]


(0.8142204743186235, 430217, 98162, 0, 0)

In [54]:
gaucoxph = DeepCoxPH()
gaucoxph.fit(gau_train_dict["x"], gau_train_dict["t"], gau_train_dict["e"],iters = 10)
risk_preds = gaucoxph.torch_model[0](torch.from_numpy(gau_train_dict["x"]))
risk_preds = risk_preds.detach().numpy()
concordance_index_censored(gau_train_dict["e"].astype(bool),gau_train_dict["t"],risk_preds.ravel())

 30%|███       | 3/10 [00:00<00:00, 25.67it/s]


(0.5203551473820636, 913804, 842312, 0, 1)

In [7]:
from bayesian_torch.models.dnn_to_bnn import dnn_to_bnn, get_kl_loss
const_bnn_prior_parameters = {
        "prior_mu": 0.0,
        "prior_sigma": 1.0,
        "posterior_mu_init": 0.0,
        "posterior_rho_init": -3.0,
        "type": "Reparameterization",  # Flipout or Reparameterization
        "moped_enable": False,  # True to initialize mu/sigma from the pretrained dnn weights
        "moped_delta": 0.5,
}

In [8]:
def partial_ll_loss(lrisks, tb, eb, eps=1e-3):

    tb = tb + eps*np.random.random(len(tb))
    sindex = np.argsort(-tb)

    tb = tb[sindex]
    eb = eb[sindex]

    lrisks = lrisks[sindex]
    lrisksdenom = torch.logcumsumexp(lrisks, dim = 0)

    plls = lrisks - lrisksdenom
    pll = plls[eb == 1]

    pll = torch.sum(pll)

    return -pll

In [21]:
batchsize = 256
learning_rate = 1e-3

x = torch.from_numpy(lin_train_dict["x"]).float()
t = torch.from_numpy(lin_train_dict["t"]).float()
e = torch.from_numpy(lin_train_dict["e"]).float()




In [10]:
CPH_torch = lincoxph.torch_model[0]
dnn_to_bnn(CPH_torch, const_bnn_prior_parameters)

In [13]:
optimizer = torch.optim.Adam(CPH_torch.parameters(), learning_rate)

output = CPH_torch(xb)
kl = get_kl_loss(CPH_torch)
ce_loss = partial_ll_loss(output,tb,eb)
loss = ce_loss + kl / batchsize

loss.backward()
optimizer.step()

In [15]:
x_train.shape[0]

5000

In [16]:
epochs = 10

In [ ]:
def train_step_bnn_cph(cph_bnn, x, t, e, optimizer, batchsize=256):
    
    n = x.shape[0]

    batches = (n // batchsize) + 1

    epoch_loss = 0

    for i in range(batches):

        xb = x[i*batchsize:(i+1)*batchsize]
        tb = t[i*batchsize:(i+1)*batchsize]
        eb = e[i*batchsize:(i+1)*batchsize]

        # Training Step

        torch.enable_grad()
        optimizer.zero_grad()

        output = cph_bnn(xb)
        kl = get_kl_loss(cph_bnn)
        ce_loss = partial_ll_loss(output,tb,eb)
        loss = ce_loss + kl / batchsize


        loss.backward()
        optimizer.step()


        epoch_loss += float(loss)
    step_loss = epoch_loss/n
    return step_loss



In [ ]:
def train_cph_bnn(cph_bnn, train_data, epochs=50,
               patience=3, batchsize=256, lr=1e-3, debug=False,
               random_state=0, return_losses=False):

    torch.manual_seed(random_state)
    np.random.seed(random_state)

    xt, tt, et = train_data
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)


    valc = np.inf
    patience_ = 0

    losses = []

    for epoch in tqdm(range(epochs)):

        _ =train_step_bnn_cph(cph_bnn, xt, tt, et, optimizer, batchsize)
        
        #test_step_still has to be implemented
        #valcn = test_step(model, xv, tv_, ev_)
        valcn = _

        losses.append(valcn)


        if valcn > valc:
            patience_ += 1
        else:
            patience_ = 0

        if patience_ == patience:
            break

        valc = valcn

    if return_losses:
        return (model, breslow_spline), losses
    else:
        return (model, breslow_spline)


In [22]:
for epoch in tqdm(range(epochs)):
    
    optimizer = torch.optim.Adam(CPH_torch.parameters(), learning_rate)


    n = x.shape[0]

    batches = (n // batchsize) + 1

    epoch_loss = 0

    for i in range(batches):

        xb = x[i*batchsize:(i+1)*batchsize]
        tb = t[i*batchsize:(i+1)*batchsize]
        eb = e[i*batchsize:(i+1)*batchsize]

        # Training Step

        torch.enable_grad()
        optimizer.zero_grad()

        output = CPH_torch(xb)
        kl = get_kl_loss(CPH_torch)
        ce_loss = partial_ll_loss(output,tb,eb)
        loss = ce_loss + kl / batchsize


        loss.backward()
        optimizer.step()


        epoch_loss += float(loss)
    step_loss = epoch_loss/n
    print("epoch completed")




100%|██████████| 10/10 [00:00<00:00, 54.73it/s]

epoch completed
epoch completed
epoch completed
epoch completed
epoch completed
epoch completed
epoch completed
epoch completed
epoch completed
epoch completed


In [ ]:

class BDNN_Zhang(nn.Module):
  '''
    DNN from the paper
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(4, 8),
      nn.Tanh(),
      nn.Linear(8, 4),
      nn.Tanh(),
      nn.Linear(4, 1),
      nn.Sigmoid(),
      nn.Linear(1, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [ ]:
def train_torch_net(trainloader,neural_net,loss_function, n_epochs = 50):

    optimizer = torch.optim.Adam(neural_net.parameters(), lr=1e-4)

    # Run the training loop
    for epoch in range(0, n_epochs):  # 5 epochs at maximum

        # Set current loss value
        loss_per_batch = []

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):

            # Get and prepare inputs
            inputs, targets = data
            inputs, targets = inputs.float(), targets.float()
            #targets = targets.reshape((targets.shape[0], 1))

            # Zero the gradients
            optimizer.zero_grad()
            # Perform forward pass
            outputs = neural_net(inputs)
            # Compute loss
            loss = loss_function(outputs, targets)
            # Perform backward pass
            loss.backward()
            # Perform optimization
            optimizer.step()

            loss_per_batch.append(loss.item())

    return neural_net, loss_per_batch

In [37]:
train_dataset_fp = '../DeepSurv/notebooks/example_data.csv'
train_df = pd.read_csv(train_dataset_fp)
train_df.head()

,Variable_1,Variable_2,Variable_3,Variable_4,Event,Time
0,0,3,2,4.6,1,43
1,0,2,0,1.6,0,52
2,0,3,0,3.5,1,73
3,0,3,1,5.1,0,51
4,0,2,0,1.7,0,51


In [38]:
ds_datasets = stg_utils.load_cox_gaussian_data()

